In [1]:
%load_ext autoreload
%autoreload 2
from doclassifier import *

In [2]:
path_name = '/Users/jordanvalansi/Downloads/Thoughts' 
sents_df = pd.DataFrame(get_texts(path_name))

/Users/jordanvalansi/Downloads/Thoughts/.DS_Store Not added
/Users/jordanvalansi/Downloads/Thoughts/logical fallacies.xlsx Not added
/Users/jordanvalansi/Downloads/Thoughts/Untitled drawing(2).jpg Not added
/Users/jordanvalansi/Downloads/Thoughts/Untitled drawing(1).jpg Not added
/Users/jordanvalansi/Downloads/Thoughts/Untitled spreadsheet.xlsx Not added
/Users/jordanvalansi/Downloads/Thoughts/Untitled drawing.jpg Not added
/Users/jordanvalansi/Downloads/Thoughts/Letter Origin.jpg Not added


In [3]:
sents_df.index.name = 'fname'
sents_df.columns = ['he_sents']

In [4]:
sents_df.head()

,he_sents
fname,
איך ללמוד.docx,מושגים
איך ללמוד.docx,קשרים בין מושגים
טענה_.docx,טענה:
טענה_.docx,או שאפשר לחשוב על זה כמו באפט
טענה_.docx,או שאפשר לעשות כל סטארט אפ


In [87]:
len(sents_df)

5084

In [65]:
sents_dict = sents_df.reset_index().groupby('fname')['he_sents'].apply(lambda x: [i for i in x]).to_dict()

In [11]:
he_texts = pd.DataFrame(sents_df.groupby(level=0)['he_sents'].apply(lambda x: '\n'.join(x)))

In [9]:
en_texts_df = pd.read_csv('data/en_sents_textblob.csv', names=['fname', 'en_sents'])

In [12]:
en_texts_df = en_texts_df.set_index('fname')

In [13]:
en_texts_df['he_texts'] = he_texts

In [21]:
en_texts_df = en_texts_df.dropna()

In [23]:

def split_sents(x):
    try:
        en_sents = list(filter(None, re.split('\n+', x['en_sents'])))
        he_sents = list(filter(None, re.split('\n+', x['he_texts'])))
        return pd.Series(list(zip(en_sents,he_sents)))
    except:
        return None


sent_df = pd.melt(en_texts_df.apply(split_sents , axis=1).reset_index(), id_vars='fname').sort(['fname', 'variable']).dropna().set_index('fname')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  # This is added back by InteractiveShellApp.init_path()


In [25]:
sent_df = sent_df.apply(lambda x: pd.Series([x['value'][0],x['value'][1]], index=['en', 'he']), axis=1)

In [206]:
sent_df.to_csv('data/sents_textblob.csv')

In [51]:
sent_df ['topic'] = sent_df.apply(lambda x: get_top_words(tokenize(x['en'], 'en'))[0], axis=1)

In [60]:
sent_df.head()

,en,he,topic,nn
fname,,,,
2nd law.docx,Assumptions:,Assumptions:,assumptions,assumption
2nd law.docx,The particle fits through the doorway with eps...,The particle fits through the doorway with eps...,epsilon,epsilon
2nd law.docx,This means that the probability that a particl...,This means that the probability that a particl...,epsilon,epsilon
2nd law.docx,The door opens up to 90 degrees to one side only.,The door opens up to 90 degrees to one side only.,opens,closes
2nd law.docx,The door’s mass is negligible (does not change...,The door’s mass is negligible (does not change...,negligible,effect


In [55]:
len(sent_df['topic'].unique())

2518

In [26]:
words = [word for text in sent_df['en'] for word in tokenize(text, 'en')]

In [27]:
count_df = get_tfidf(words)

/Users/jordanvalansi/projects/notebooks/doclassifier.py:286: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  count_df= count_df.sort('tfidf', ascending=False)


In [29]:
import gensim.downloader as api

In [30]:
model = api.load("fasttext-wiki-news-subwords-300")

In [31]:
count_df = count_df[count_df['count']>=3]

In [33]:
words_vecs = pd.Series(count_df.index).apply(lambda x: pd.Series(model.wv[x]) if x in model.wv else pd.Series(model.wv['.']))
words_vecs.index = count_df.index

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [340]:
count_df['nn'] = get_nns(words_vecs)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [357]:
topics = [k for k,v in collections.Counter(count_df['nn']).items() if v>=3]

In [72]:
topic_vecs = sent_df['nn'].apply(lambda x: pd.Series(model.wv[x]) if x in model.wv else pd.Series(model.wv['.']))
topic_vecs.index = sent_df['nn']


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [73]:
sent_df['nn'] = get_nns(topic_vecs)


In [74]:
len(sent_df['nn'].unique())

1274

In [79]:
# pd.DataFrame(list(model.vocab.values()), index=list(model.vocab.keys()), columns=['vocab'])

In [47]:
model.most_similar('mail')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('mails', 0.8362846970558167),
 ('mail-', 0.795038640499115),
 ('mailing', 0.7847987413406372),
 ('-mail', 0.7756238579750061),
 ('mail-box', 0.77336186170578),
 ('V-mail', 0.7717906832695007),
 ('mailer', 0.7670629620552063),
 ('mass-mail', 0.7626809477806091),
 ('junk-mail', 0.7585399150848389),
 ('mail.', 0.7544159889221191)]